In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [54]:
df = pd.read_csv('train.csv')

In [55]:
df.shape

(404290, 6)

In [56]:
# new_df = df.sample(30000)
# new_df = df.sample(300)
new_df = df

In [57]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
356118,356118,485393,485394,What is the problems of trade manufactured goo...,Which voting groups did Hilary do better than ...,0
309345,309345,433321,30782,Do spammers make money?,How can you look at someone's private Instagra...,0
213099,213099,318345,318346,Why do corrupt and evil people flourish while ...,"Is Karma real? If so, then why do we see bad p...",1
122902,122902,198822,198823,Where was the wheel invented?,When was the wheel invented?,0
205461,205461,308611,308612,Is it too late to learn playing drums at the a...,Is it too late to learn drums at age 34?,0


In [58]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q
    

In [59]:
preprocess("I've already! wasn't <b>done</b>?")

'i have already  was not done'

In [60]:
new_df['question1'] = new_df['question1'].apply(preprocess)
new_df['question2'] = new_df['question2'].apply(preprocess)

In [61]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
356118,356118,485393,485394,what is the problems of trade manufactured goo...,which voting groups did hilary do better than ...,0
309345,309345,433321,30782,do spammers make money,how can you look at someone s private instagra...,0
213099,213099,318345,318346,why do corrupt and evil people flourish while ...,is karma real if so then why do we see bad p...,1
122902,122902,198822,198823,where was the wheel invented,when was the wheel invented,0
205461,205461,308611,308612,is it too late to learn playing drums at the a...,is it too late to learn drums at age 34,0


In [62]:
new_df['q1_len'] = new_df['question1'].str.len() 
new_df['q2_len'] = new_df['question2'].str.len()

In [63]:
new_df['q1_num_words'] = new_df['question1'].apply(lambda row: len(row.split(" ")))
new_df['q2_num_words'] = new_df['question2'].apply(lambda row: len(row.split(" ")))
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words
356118,356118,485393,485394,what is the problems of trade manufactured goo...,which voting groups did hilary do better than ...,0,91,86,13,14
309345,309345,433321,30782,do spammers make money,how can you look at someone s private instagra...,0,22,78,4,13
213099,213099,318345,318346,why do corrupt and evil people flourish while ...,is karma real if so then why do we see bad p...,1,69,117,12,27
122902,122902,198822,198823,where was the wheel invented,when was the wheel invented,0,28,27,5,5
205461,205461,308611,308612,is it too late to learn playing drums at the a...,is it too late to learn drums at age 34,0,54,39,13,10


In [64]:
def common_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return len(w1 & w2)

In [65]:
new_df['word_common'] = new_df.apply(common_words, axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common
356118,356118,485393,485394,what is the problems of trade manufactured goo...,which voting groups did hilary do better than ...,0,91,86,13,14,1
309345,309345,433321,30782,do spammers make money,how can you look at someone s private instagra...,0,22,78,4,13,0
213099,213099,318345,318346,why do corrupt and evil people flourish while ...,is karma real if so then why do we see bad p...,1,69,117,12,27,8
122902,122902,198822,198823,where was the wheel invented,when was the wheel invented,0,28,27,5,5,4
205461,205461,308611,308612,is it too late to learn playing drums at the a...,is it too late to learn drums at age 34,0,54,39,13,10,9


In [66]:
def total_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return (len(w1) + len(w2))

In [67]:
new_df['word_total'] = new_df.apply(total_words, axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total
356118,356118,485393,485394,what is the problems of trade manufactured goo...,which voting groups did hilary do better than ...,0,91,86,13,14,1,27
309345,309345,433321,30782,do spammers make money,how can you look at someone s private instagra...,0,22,78,4,13,0,17
213099,213099,318345,318346,why do corrupt and evil people flourish while ...,is karma real if so then why do we see bad p...,1,69,117,12,27,8,37
122902,122902,198822,198823,where was the wheel invented,when was the wheel invented,0,28,27,5,5,4,10
205461,205461,308611,308612,is it too late to learn playing drums at the a...,is it too late to learn drums at age 34,0,54,39,13,10,9,23


In [68]:
new_df['word_share'] = round(new_df['word_common']/new_df['word_total'],2)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share
356118,356118,485393,485394,what is the problems of trade manufactured goo...,which voting groups did hilary do better than ...,0,91,86,13,14,1,27,0.04
309345,309345,433321,30782,do spammers make money,how can you look at someone s private instagra...,0,22,78,4,13,0,17,0.00
213099,213099,318345,318346,why do corrupt and evil people flourish while ...,is karma real if so then why do we see bad p...,1,69,117,12,27,8,37,0.22
122902,122902,198822,198823,where was the wheel invented,when was the wheel invented,0,28,27,5,5,4,10,0.40
205461,205461,308611,308612,is it too late to learn playing drums at the a...,is it too late to learn drums at age 34,0,54,39,13,10,9,23,0.39


In [69]:
# Advanced Features
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


def fetch_token_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mutahhir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
token_features = new_df.apply(fetch_token_features, axis=1)

new_df["cwc_min"]       = list(map(lambda x: x[0], token_features))
new_df["cwc_max"]       = list(map(lambda x: x[1], token_features))
new_df["csc_min"]       = list(map(lambda x: x[2], token_features))
new_df["csc_max"]       = list(map(lambda x: x[3], token_features))
new_df["ctc_min"]       = list(map(lambda x: x[4], token_features))
new_df["ctc_max"]       = list(map(lambda x: x[5], token_features))
new_df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
new_df["first_word_eq"] = list(map(lambda x: x[7], token_features))

In [71]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,word_total,word_share,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq
356118,356118,485393,485394,what is the problems of trade manufactured goo...,which voting groups did hilary do better than ...,0,91,86,13,14,...,27,0.04,0.000000,0.000000,0.166664,0.166664,0.076922,0.071428,0,0
309345,309345,433321,30782,do spammers make money,how can you look at someone s private instagra...,0,22,78,4,13,...,17,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0
213099,213099,318345,318346,why do corrupt and evil people flourish while ...,is karma real if so then why do we see bad p...,1,69,117,12,27,...,37,0.22,0.571420,0.333331,0.999975,0.307690,0.666661,0.319999,0,0
122902,122902,198822,198823,where was the wheel invented,when was the wheel invented,0,28,27,5,5,...,10,0.40,0.999950,0.999950,0.666644,0.666644,0.799984,0.799984,1,0
205461,205461,308611,308612,is it too late to learn playing drums at the a...,is it too late to learn drums at age 34,0,54,39,13,10,...,23,0.39,0.799984,0.666656,0.999980,0.714276,0.899991,0.692302,0,1


In [72]:
import distance

def fetch_length_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    length_features = [0.0]*3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(distance.lcsubstrings(q1, q2))
    if len(strs)==0:
        length_features[2] = 0
    else:
        length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    
    return length_features
    

In [73]:
length_features = new_df.apply(fetch_length_features, axis=1)

new_df['abs_len_diff'] = list(map(lambda x: x[0], length_features))
new_df['mean_len'] = list(map(lambda x: x[1], length_features))
new_df['longest_substr_ratio'] = list(map(lambda x: x[2], length_features))

In [74]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio
356118,356118,485393,485394,what is the problems of trade manufactured goo...,which voting groups did hilary do better than ...,0,91,86,13,14,...,0.000000,0.166664,0.166664,0.076922,0.071428,0,0,1,13.5,0.057471
309345,309345,433321,30782,do spammers make money,how can you look at someone s private instagra...,0,22,78,4,13,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,9,8.5,0.130435
213099,213099,318345,318346,why do corrupt and evil people flourish while ...,is karma real if so then why do we see bad p...,1,69,117,12,27,...,0.333331,0.999975,0.307690,0.666661,0.319999,0,0,13,18.5,0.128571
122902,122902,198822,198823,where was the wheel invented,when was the wheel invented,0,28,27,5,5,...,0.999950,0.666644,0.666644,0.799984,0.799984,1,0,0,5.0,0.821429
205461,205461,308611,308612,is it too late to learn playing drums at the a...,is it too late to learn drums at age 34,0,54,39,13,10,...,0.666656,0.999980,0.714276,0.899991,0.692302,0,1,3,11.5,0.600000


In [75]:
# Fuzzy Features
from fuzzywuzzy import fuzz

def fetch_fuzzy_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [76]:
fuzzy_features = new_df.apply(fetch_fuzzy_features, axis=1)

# Creating new feature columns for fuzzy features
new_df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
new_df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
new_df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
new_df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features))

In [77]:
print(new_df.shape)
new_df.head()

(300, 28)


,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
356118,356118,485393,485394,what is the problems of trade manufactured goo...,which voting groups did hilary do better than ...,0,91,86,13,14,...,0.071428,0,0,1,13.5,0.057471,21,29,38,28
309345,309345,433321,30782,do spammers make money,how can you look at someone s private instagra...,0,22,78,4,13,...,0.000000,0,0,9,8.5,0.130435,16,36,22,22
213099,213099,318345,318346,why do corrupt and evil people flourish while ...,is karma real if so then why do we see bad p...,1,69,117,12,27,...,0.319999,0,0,13,18.5,0.128571,32,45,52,78
122902,122902,198822,198823,where was the wheel invented,when was the wheel invented,0,28,27,5,5,...,0.799984,1,0,0,5.0,0.821429,95,93,95,95
205461,205461,308611,308612,is it too late to learn playing drums at the a...,is it too late to learn drums at age 34,0,54,39,13,10,...,0.692302,0,1,3,11.5,0.600000,80,79,80,96


In [78]:
# sns.pairplot(new_df[['ctc_min', 'cwc_min', 'csc_min', 'is_duplicate']],hue='is_duplicate')

In [79]:
# sns.pairplot(new_df[['ctc_max', 'cwc_max', 'csc_max', 'is_duplicate']],hue='is_duplicate')

In [80]:
# sns.pairplot(new_df[['last_word_eq', 'first_word_eq', 'is_duplicate']],hue='is_duplicate')

In [81]:
# sns.pairplot(new_df[['mean_len', 'abs_len_diff','longest_substr_ratio', 'is_duplicate']],hue='is_duplicate')

In [82]:
# sns.pairplot(new_df[['fuzz_ratio', 'fuzz_partial_ratio','token_sort_ratio','token_set_ratio', 'is_duplicate']],hue='is_duplicate')

In [83]:
# # Using TSNE for Dimentionality reduction for 15 Features(Generated after cleaning the data) to 3 dimention

# from sklearn.preprocessing import MinMaxScaler

# X = MinMaxScaler().fit_transform(new_df[['cwc_min', 'cwc_max', 'csc_min', 'csc_max' , 'ctc_min' , 'ctc_max' , 'last_word_eq', 'first_word_eq' , 'abs_len_diff' , 'mean_len' , 'token_set_ratio' , 'token_sort_ratio' ,  'fuzz_ratio' , 'fuzz_partial_ratio' , 'longest_substr_ratio']])
# y = new_df['is_duplicate'].values

In [84]:
# from sklearn.manifold import TSNE

# tsne2d = TSNE(
#     n_components=2,
#     init='random', # pca
#     random_state=101,
#     method='barnes_hut',
#     n_iter=1000,
#     verbose=2,
#     angle=0.5
# ).fit_transform(X)

In [85]:
# x_df = pd.DataFrame({'x':tsne2d[:,0], 'y':tsne2d[:,1] ,'label':y})

# # draw the plot in appropriate place in the grid
# sns.lmplot(data=x_df, x='x', y='y', hue='label', fit_reg=False, palette="Set1",markers=['s','o'])


In [86]:
# tsne3d = TSNE(
#     n_components=3,
#     init='random', # pca
#     random_state=101,
#     method='barnes_hut',
#     n_iter=1000,
#     verbose=2,
#     angle=0.5
# ).fit_transform(X)

In [87]:
# import plotly.graph_objs as go
# import plotly.tools as tls
# import plotly.offline as py
# py.init_notebook_mode(connected=True)

# trace1 = go.Scatter3d(
#     x=tsne3d[:,0],
#     y=tsne3d[:,1],
#     z=tsne3d[:,2],
#     mode='markers',
#     marker=dict(
#         sizemode='diameter',
#         color = y,
#         colorscale = 'Portland',
#         colorbar = dict(title = 'duplicate'),
#         line=dict(color='rgb(255, 255, 255)'),
#         opacity=0.75
#     )
# )

# data=[trace1]
# layout=dict(height=800, width=800, title='3d embedding with engineered features')
# fig=dict(data=data, layout=layout)
# py.iplot(fig, filename='3DBubble')

In [88]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
356118,what is the problems of trade manufactured goo...,which voting groups did hilary do better than ...
309345,do spammers make money,how can you look at someone s private instagra...
213099,why do corrupt and evil people flourish while ...,is karma real if so then why do we see bad p...
122902,where was the wheel invented,when was the wheel invented
205461,is it too late to learn playing drums at the a...,is it too late to learn drums at age 34


In [89]:
final_df = new_df.drop(columns=['id','qid1','qid2','question1','question2'])
print(final_df.shape)
final_df.head()

(300, 23)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
356118,0,91,86,13,14,1,27,0.04,0.000000,0.000000,...,0.071428,0,0,1,13.5,0.057471,21,29,38,28
309345,0,22,78,4,13,0,17,0.00,0.000000,0.000000,...,0.000000,0,0,9,8.5,0.130435,16,36,22,22
213099,1,69,117,12,27,8,37,0.22,0.571420,0.333331,...,0.319999,0,0,13,18.5,0.128571,32,45,52,78
122902,0,28,27,5,5,4,10,0.40,0.999950,0.999950,...,0.799984,1,0,0,5.0,0.821429,95,93,95,95
205461,0,54,39,13,10,9,23,0.39,0.799984,0.666656,...,0.692302,0,1,3,11.5,0.600000,80,79,80,96


In [90]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [91]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(300, 3444)

In [92]:
final_df = pd.concat([final_df, temp_df], axis=1)
print(final_df.shape)
final_df.head()

(300, 3467)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721
356118,0,91,86,13,14,1,27,0.04,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
309345,0,22,78,4,13,0,17,0.00,0.000000,0.000000,...,1,0,0,0,0,0,0,0,0,0
213099,1,69,117,12,27,8,37,0.22,0.571420,0.333331,...,0,0,0,0,0,0,0,0,0,0
122902,0,28,27,5,5,4,10,0.40,0.999950,0.999950,...,0,0,0,0,0,0,0,0,0,0
205461,0,54,39,13,10,9,23,0.39,0.799984,0.666656,...,0,0,0,0,0,0,0,0,0,0


In [93]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=1)

In [94]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.65

In [95]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred1 = xgb.predict(X_test)
accuracy_score(y_test,y_pred1)

0.6

In [96]:
from sklearn.metrics import confusion_matrix

In [97]:
# for random forest model
confusion_matrix(y_test,y_pred)

array([[26, 19],
       [ 2, 13]], dtype=int64)

In [98]:
# for xgboost model
confusion_matrix(y_test,y_pred1)

array([[28, 17],
       [ 7,  8]], dtype=int64)

In [99]:
def test_common_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))    
    return len(w1 & w2)

In [100]:
def test_total_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))    
    return (len(w1) + len(w2))

In [101]:
def test_fetch_token_features(q1,q2):
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features


In [102]:
def test_fetch_length_features(q1,q2):
    
    length_features = [0.0]*3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(distance.lcsubstrings(q1, q2))
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    
    return length_features

In [103]:
def test_fetch_fuzzy_features(q1,q2):
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [104]:
def query_point_creator(q1,q2):
    
    input_query = []
    
    # preprocess
    q1 = preprocess(q1)
    q2 = preprocess(q2)
    
    # fetch basic features
    input_query.append(len(q1))
    input_query.append(len(q2))
    
    input_query.append(len(q1.split(" ")))
    input_query.append(len(q2.split(" ")))
    
    input_query.append(test_common_words(q1,q2))
    input_query.append(test_total_words(q1,q2))
    input_query.append(round(test_common_words(q1,q2)/test_total_words(q1,q2),2))
    
    # fetch token features
    token_features = test_fetch_token_features(q1,q2)
    input_query.extend(token_features)
    
    # fetch length based features
    length_features = test_fetch_length_features(q1,q2)
    input_query.extend(length_features)
    
    # fetch fuzzy features
    fuzzy_features = test_fetch_fuzzy_features(q1,q2)
    input_query.extend(fuzzy_features)
    
    # bow feature for q1
    q1_bow = cv.transform([q1]).toarray()
    
    # bow feature for q2
    q2_bow = cv.transform([q2]).toarray()
    
    
    
    return np.hstack((np.array(input_query).reshape(1,22),q1_bow,q2_bow))

In [105]:
q1 = 'Where is the capital of India?'
q2 = 'What is the current capital of Pakistan?'
q3 = 'Which city serves as the capital of India?'
q4 = 'What is the business capital of India?'

In [106]:
rf.predict(query_point_creator(q1,q4))

array([1], dtype=int64)

In [107]:
cv

CountVectorizer(max_features=3000)

In [108]:
import pickle

pickle.dump(rf,open('model.pkl','wb'))
pickle.dump(cv,open('cv.pkl','wb'))